In [2]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import cv2
import sys
from tensorflow.keras import layers, models # type: ignore
from sklearn.model_selection import train_test_split
import pickle

In [2]:
# Function to train the CNN model (or load a pre-trained one)
def build_and_train_model():
    (x_train_full, y_train_full), (x_test, y_test) = tf.keras.datasets.mnist.load_data() #Load Data
    x_train_full = x_train_full.astype('float32') / 255.0 #Normalize //from 0 to 1
    x_test = x_test.astype('float32') / 255.0 #Normalize
    x_train, x_val, y_train, y_val = train_test_split(x_train_full, y_train_full, test_size=0.2, random_state=42)

    x_train = x_train.reshape(-1, 28, 28, 1)
    x_val = x_val.reshape(-1, 28, 28, 1)
    x_test = x_test.reshape(-1, 28, 28, 1)

    # Build the CNN model
    model = models.Sequential([
        layers.Conv2D(64, (3, 3), activation='relu', input_shape=(28, 28, 1), padding='same'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(256, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),
        layers.Flatten(),
        layers.Dense(512, activation='relu'),
        layers.Dropout(0.6), #to prevent overfitting
        layers.Dense(10, activation='softmax')  # 10 classes for digits 0-9
    ])

    # Compile the model
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

    # Train the model
    model.fit(x_train, y_train, epochs=10, batch_size=32, validation_data=(x_val, y_val))

    # Save the model
    model.save('attendance_digit_model.h5')

In [3]:
from tensorflow.keras.models import load_model 
#build_and_train_model()
model = load_model('./attendance_digit_model.h5')

In [4]:
from PIL import Image

def load_and_preprocess_image(path):
    img = Image.open(path)
    #.convert('L')           # Convert to grayscale
    #img = img.resize((28, 28))                    # Resize to 28x28
    # img_array = np.array(img) / 255.0             # Normalize
    # img_array = 1 - img_array                     # Invert colors if white digit on black bg
    # img_array = img_array.reshape(1, 28, 28, 1)   # Add batch and channel dims
    return img

def load_images_from_folder(folder):
    images = []
    filenames = []
    for filename in sorted(os.listdir(folder)):
        if filename.lower().endswith((".png", ".jpg", ".jpeg")):
            path = os.path.join(folder, filename)
            img = Image.open(path).convert("L")
            # img = np.array(img) / 255.0  # normalize
            # img = 1 - img  # invert
            # img = img.reshape(28, 28, 1)
            images.append(img)
            filenames.append(filename)
    return np.stack(images), filenames  # <- stack into one tensor


In [7]:
import os
# Load all images
image_folder = "../images/final_clean_cells/"
images, filenames = load_images_from_folder(image_folder)

# # Function to display images
# def display_images(images, titles, rows, cols, figsize=(15, 5)):
#     fig, axes = plt.subplots(rows, cols, figsize=figsize)
#     for i, (img, title) in enumerate(zip(images, titles)):
#         ax = axes[i//cols, i%cols] if rows > 1 else axes[i]
#         ax.imshow(img, cmap='gray')
#         ax.set_title(title)
#         ax.axis('off')
#     plt.tight_layout()
#     plt.show()

# # Display first 5 images
# print("First 5 images:")
# display_images(images[:5], [f"Image {i+1}" for i in range(5)], 1, 5)

# # Display last 5 images
# print("\nLast 5 images:")
# display_images(images[-5:], [f"Image {len(images)-4+i}" for i in range(5)], 1, 5)

# Predict on the full batch
prediction = model.predict(images)
predicted_digits = np.argmax(prediction, axis=1)

# Output results
for fname, digit in zip(filenames, predicted_digits):
    print(f"{fname} ➤ {digit}")

6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
id_cell_001.png ➤ 0
id_cell_002.png ➤ 0
id_cell_003.png ➤ 0
id_cell_004.png ➤ 0
id_cell_005.png ➤ 8
id_cell_006.png ➤ 0
id_cell_007.png ➤ 0
id_cell_008.png ➤ 0
id_cell_009.png ➤ 0
id_cell_010.png ➤ 0
id_cell_011.png ➤ 0
id_cell_012.png ➤ 8
id_cell_013.png ➤ 0
id_cell_014.png ➤ 0
id_cell_015.png ➤ 0
id_cell_016.png ➤ 0
id_cell_017.png ➤ 1
id_cell_018.png ➤ 1
id_cell_019.png ➤ 1
id_cell_020.png ➤ 1
id_cell_021.png ➤ 1
id_cell_022.png ➤ 1
id_cell_023.png ➤ 1
id_cell_024.png ➤ 7
id_cell_025.png ➤ 7
id_cell_026.png ➤ 1
id_cell_027.png ➤ 1
id_cell_028.png ➤ 1
id_cell_029.png ➤ 1
id_cell_030.png ➤ 1
id_cell_031.png ➤ 1
id_cell_032.png ➤ 1
id_cell_033.png ➤ 2
id_cell_034.png ➤ 2
id_cell_035.png ➤ 2
id_cell_036.png ➤ 2
id_cell_037.png ➤ 2
id_cell_038.png ➤ 2
id_cell_039.png ➤ 2
id_cell_040.png ➤ 2
id_cell_041.png ➤ 2
id_cell_042.png ➤ 2
id_cell_043.png ➤ 2
id_cell_044.png ➤ 2
id_cell_045.png ➤ 2
id_cell_046.png ➤ 2
id_cell_047.png ➤ 2
id_cell_048.png ➤ 2
id